In [2]:
# Import packages
import pandas as pd

In [15]:
# Import Urban Institute Household Conditions data
ui_df = pd.read_csv('NHGIS_District_data.csv')

# Drop extraneous columns & rename columns
ui_df.drop(['School ID',
            'Single Parent Margin of Error', 
            'Vulnerable Job Margin of Error', 
            'HH With Crowded Conditions Margin of Error', 
            'No Computer or Internet Margin of Error',
            'Children with Disability Margin of Error',
            'Linguistically Isolated Children Margin of Error'], 
           axis=1, inplace=True)
ui_df.rename(columns={"Geographic School District": "School District"}, inplace=True)

# Replace state names with abbreviations
states = {'Alaska': 'AK', 'Alabama': 'AL', 'Arkansas': 'AR', 'American Samoa': 'AS', 'Arizona': 'AZ', 'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'District of Columbia': 'DC', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA', 'Guam': 'GU', 'Hawaii': 'HI', 'Iowa': 'IA', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Massachusetts': 'MA', 'Maryland': 'MD', 'Maine': 'ME', 'Michigan': 'MI', 'Minnesota': 'MN', 'Missouri': 'MO', 'Northern Mariana Islands': 'MP', 'Mississippi': 'MS', 'Montana': 'MT', 'National': 'NA', 'North Carolina': 'NC', 'North Dakota': 'ND', 'Nebraska': 'NE', 'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'Nevada': 'NV', 'New York': 'NY', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA', 'Puerto Rico': 'PR', 'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT', 'Virginia': 'VA', 'Virgin Islands': 'VI', 'Vermont': 'VT', 'Washington': 'WA', 'Wisconsin': 'WI', 'West Virginia': 'WV', 'Wyoming': 'WY'}
ui_df['State'] = ui_df['State'].replace(states)

print('School districts: ' + str(ui_df.shape[0]))

School districts: 13314


In [16]:
# Import School District/County FIPS data & rename columns
districts = pd.read_csv('sdlist-22.csv', header=1)
districts.rename(columns={"State Postal Code": "State", "School District Name": "School District", "County Names": "County"}, inplace=True)

# Merge School District/County FIPS data with Urban Institute data
ui_df = pd.merge(ui_df, districts, how='inner', 
                  left_on=['State','School District'], 
                  right_on = ['State','School District'])

# Drop duplicate state/district/county combination rows
ui_df.drop_duplicates(subset=['State', 'School District', 'County'], inplace=True)
print('Unique county/school district combinations: ' + str(ui_df.shape[0]))

Unique county/school district combinations: 18895


In [17]:
# TODO later: Combine all districts for each county proportionally instead of dropping duplicates
ui_df.drop_duplicates(subset=['State', 'County'], inplace=True)
print('Counties with at least one school district: ' + str(ui_df.shape[0]))

Counties with at least one school district: 3138


In [18]:
# Combine State & County FIPS into full FIPS ID then drop State & COunty FIPS
ui_df['FIPS'] = ui_df['State FIPS'].astype(str).str.zfill(2) + ui_df['County FIPS'].astype(str).str.zfill(3)
ui_df.drop(['State FIPS', 'County FIPS'], axis=1, inplace=True)
ui_df.head(1)

State              School District Children 5-17 (SAIPE Estimate)  \
0    AL  Fort Rucker School District                            985   

  % Poverty (SAIPE Estimate) % Single Parent Estimate  \
0                        12%                       5%   

  % HHs With Vulnerable Job Estimate % Crowded Conditions Estimate  \
0                                33%                            3%   

  % No Computer or Internet Estimate % Children with Disability  \
0                                 2%                         3%   

  % Linguistically Isolated Children  District ID Number         County   FIPS  
0                                 0%                   1  Coffee County  01031

In [19]:
final_df = ui_df.copy()

# Drop extraneous columns
final_df.drop(['School District', 'Children 5-17 (SAIPE Estimate)', 'District ID Number'], axis=1, inplace=True)
final_df.rename(columns={"FIPS": "id", "State": "state", "County": "county"}, inplace=True)
attributes = ['% Poverty (SAIPE Estimate)', '% Single Parent Estimate', '% HHs With Vulnerable Job Estimate', '% Crowded Conditions Estimate', '% No Computer or Internet Estimate', '% Children with Disability', '% Linguistically Isolated Children']
for a in attributes:
    final_df[a] = pd.to_numeric(final_df[a].str.replace('%', ''))
final_df.to_csv('data.csv', index=False)

In [8]:
# Export each attribute as a separate CSV with IDs for use in Choropleth visualizations
attributes = ['% Poverty (SAIPE Estimate)', '% Single Parent Estimate', '% HHs With Vulnerable Job Estimate', '% Crowded Conditions Estimate', '% No Computer or Internet Estimate', '% Children with Disability', '% Linguistically Isolated Children']
ids = ['State', 'County', 'FIPS']

temp_df = pd.DataFrame()
for a in attributes:
    temp_df = ui_df[[a] + ids]
    temp_df[a] = pd.to_numeric(temp_df[a].str.replace('%', ''))
    temp_df.rename(columns={"FIPS": "id", "State": "state", "County": "county", a: "rate"}, inplace=True)
    temp_df.to_csv(a + '.csv', index=False)
temp_df.head(1)

<ipython-input-8-2cf83e6bc10b>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df[a] = pd.to_numeric(temp_df[a].str.replace('%', ''))


rate state         county    id
0     0    AL  Coffee County  1031